In [1]:
import os
os.chdir("../")

In [2]:
#enity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps:int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [3]:
#configuration manager
#Config
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml,create_directory


class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifact_root])
        
    def model_trainer_config(self) ->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        create_directory([config.root_dir])
        
        data_transformation_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps)
        return data_transformation_config
 

In [4]:
import os
from TextSummarizer.logging import  logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [5]:
#components
from transformers import TrainingArguments,Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset,load_from_disk
import torch


In [6]:
class ModelTrainer:
    def __init__(self,config):
        self.config = config
    
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)
        
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        training_args = TrainingArguments(
            output_dir = self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size= self.config.per_device_train_batch_size,
            weight_decay = self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps)
        
        trainer = Trainer(
            model=model_pegasus,
            args= training_args,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"]
        )
        trainer.train()
        
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [8]:
#pipeline
try:
    config = ConfigurationManager()
    trainer_config = config.model_trainer_config()
    model_trainer = ModelTrainer(trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-11-23 19:19:28,181: INFO: common: Directory Artifacts created]
[2023-11-23 19:19:28,184: INFO: common: Directory Artifacts/model_trainer created]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss



KeyboardInterrupt



In [55]:
!pip install --upgrade accelerate